In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [ ]:
#this is the normal way of getting data
'''
df = pd.read_csv("~/data/3-Simon/promoterDHS_distalDHS_pairs.csv")
df = df.sample(n=50000)
df['promoter_len'] = df['promoterDHSend']-df['promoterDHSstart']
df['distal_len'] = df['distalDHSend'] - df['distalDHSstart']
'''
#but we'll use josh's indices which are balanced already
sample_idx = np.load("/home/pau557/BCdata_project3/SampleIdx.npz")
idx1, idx0 = sample_idx['idx1'], sample_idx['idx0']
np.concatenate([idx1, idx0])
idx10 = np.concatenate([idx1,idx0])
np.random.shuffle(idx10)
df = pd.read_csv("~/data/3-Simon/promoterDHS_distalDHS_pairs.csv")
newdf = df.iloc[idx10]
del df
df = newdf
del newdf
df['promoter_len'] = df['promoterDHSend']-df['promoterDHSstart']
df['distal_len'] = df['distalDHSend'] - df['distalDHSstart']

In [ ]:
print(df.shape)

In [ ]:
df.head()

In [ ]:
plt.figure
plt.hist(df['promoter_len'],bins=30,color='g',alpha=0.4,label='promoter')
plt.hist(df['distal_len'],bins=30,color='r',alpha=0.4, label='distal')
plt.legend()
plt.show()

In [ ]:
seqlength = 2000
df = df[(df['promoter_len']<seqlength)&(df['distal_len']<seqlength)]

In [ ]:
Y = np.zeros((df.shape[0]),dtype=int)
c = 0
for i, row in df.iterrows():
    Y[c]=int(row.linked)
    c += 1

In [ ]:
class_weight_vec = sklearn.utils.class_weight.compute_class_weight('balanced',[0.,1.],Y)

In [ ]:
print(class_weight_vec)

In [ ]:
N = df.shape[0]
encoding_promoter = np.zeros((N,seqlength*2,4),dtype=bool)
encoding_distal = np.zeros((N,seqlength*2,4),dtype=bool)

In [ ]:
from Bio import SeqIO

def chrom2seq(chrom):
    """
    Convert chromosome string "chrom" to a genomic sequence
    """

    return list(SeqIO.parse(
        "/home/pau557/data/3-Simon/hg38.analysisSet.chroms/%s.fa" % chrom, "fasta"))[0].seq.upper()

In [ ]:
chrom2seq('chr1')

In [ ]:
# list of chromosomes
CHROMS = ["chr1", "chr10", "chr11", "chr12", "chr13", "chr14", "chr15", "chr16", "chr17", 
          "chr18", "chr19", "chr2", "chr20", "chr21", "chr22", "chr3", "chr4", "chr5", "chr6", 
          "chr7", "chr8", "chr9", "chrM", "chrX", "chrY"]

# Make dictionary
CHROM2SEQ = dict(zip(CHROMS,[chrom2seq(chrom) for chrom in CHROMS]))

In [ ]:
seq = CHROM2SEQ['chr1'][925589:925790]

In [ ]:
print(seq)

In [ ]:
import numpy as np

base2row = {'A':0,'C':1,'G':2,'T':3}
def double_seq2onehot(seq):
    """
    Create a one-hot encoding of a nucleotide sequence
    """
    
    A = np.zeros((len(seq)*2,4),dtype=bool)
    
    for i, base in enumerate(seq):
        if base == 'N': continue
        A[i,base2row[base]] = True
        
    for i, base in enumerate(seq[::-1]):
        if base == 'N': continue
        A[i+len(seq),base2row[base]] = True
        
    return A

In [ ]:
c = 0
for i, row in df.iterrows():
    encoding_promoter[c,:row.promoter_len*2,:]=double_seq2onehot(CHROM2SEQ[row.chr][row.promoterDHSstart:row.promoterDHSend])
    encoding_distal[c,:row.distal_len*2,:]=double_seq2onehot(CHROM2SEQ[row.chr][row.distalDHSstart:row.distalDHSend])
    c += 1
    

In [ ]:
import keras

we'll start from a logistic regression of weather a dital region is an enhancer

In [ ]:
from keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense

In [ ]:
from keras.models import Model

In [ ]:
one_hot_encoding_promoter = Input(shape=(seqlength*2,4),name='promDHSoh') #oh is one hot
one_hot_encoding_distal = Input(shape=(seqlength*2,4),name='distalDHSoh') #oh is one hot

In [ ]:
from keras import regularizers
from keras.constraints import non_neg

In [ ]:
nb_filters = 12 #temp, we'll change it later
filter_len = 6 #temp, we'll change it later

conv_layer_distal = Conv1D(nb_filters,filter_len,padding='same',name='distalDHSconv',activation='relu',kernel_regularizer=regularizers.l1(0.0001),kernel_constraint=non_neg())(one_hot_encoding_distal)
fingerprint_distal = GlobalMaxPooling1D()(conv_layer_distal)

conv_layer_promoter = Conv1D(nb_filters,filter_len,padding='same',name='promDHSconv',activation='relu',kernel_regularizer=regularizers.l1(0.0001),kernel_constraint=non_neg())(one_hot_encoding_promoter)
fingerprint_promoter = GlobalMaxPooling1D()(conv_layer_promoter)

merged_vector = keras.layers.concatenate([fingerprint_distal,fingerprint_promoter],axis=-1)
deeplayer1 = Dense(20,activation='relu',name='deeplayer1')(merged_vector)
deeplayer2 = Dense(20,activation='relu',name='deeplayer2')(deeplayer1)

prob_association = Dense(1,activation='sigmoid',name='logistic_regression')(deeplayer2)
#one neuron logistic regression

In [ ]:
model = Model(inputs=[one_hot_encoding_distal,one_hot_encoding_promoter],outputs=prob_association)

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import Adam

In [ ]:
#this resets the model, but later on there is a box where you can load the values from a previous run
model.compile(optimizer=Adam(),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_ind, test_ind, y_train, y_test = train_test_split(range(len(Y)), Y, test_size=0.20, random_state=42)

In [ ]:
X_promoter_train = encoding_promoter[train_ind,:,:]
X_promoter_test = encoding_promoter[test_ind,:,:]

X_distal_train = encoding_distal[train_ind,:,:]
X_distal_test = encoding_distal[test_ind,:,:]

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpoint = ModelCheckpoint('pau_model', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
#if you want to continue from a saved model instead of resetting everything
from keras.models import load_model
model = load_model('pau_model')

In [ ]:
model.fit([X_distal_train,X_promoter_train], y_train, batch_size=250, epochs=30, validation_split=0.2,class_weight={0:class_weight_vec[0],1:class_weight_vec[1]},callbacks=[early_stopping,checkpoint])

In [ ]:
"""
Visualizing the distal filters
"""
filters=model.get_layer('distalDHSconv').get_weights()[0]
fig, axs = plt.subplots(3,4, figsize=(15, 6), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()
for i in range(12):
    im=axs[i].imshow(filters[:,:,i].T)
    #plt.yticks(range(4),['A','T','C','G'])
    plt.colorbar(im,ax=axs[i])
plt.title('Filters')
plt.show()

In [ ]:
"""
Visualizing the distal filters
"""
filters=model.get_layer('promDHSconv').get_weights()[0]
fig, axs = plt.subplots(3,4, figsize=(15, 6), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()
for i in range(12):
    im=axs[i].imshow(filters[:,:,i].T)
    #plt.yticks(range(4),['A','T','C','G'])
    plt.colorbar(im,ax=axs[i])
plt.title('Filters')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

In [ ]:
"""
ROC score
"""
#y_score = model.predict_proba(xd_dev, verbose=0)
print('Predicting on test data')
y_score = model.predict([X_distal_test,X_promoter_test])
score=roc_auc_score(y_test, y_score, average='macro', sample_weight=None)
print(score)

In [ ]:
import scipy.stats as ss

In [ ]:
y_score.T

In [ ]:
ss.pearsonr(y_test,np.array(y_score.T[0]))

In [ ]:
plt.figure()
plt.scatter(y_test,np.array(y_score.T[0]),alpha=0.01,marker='.')
plt.show()

In [ ]:
# look at Keras Merge for enhancers and promoters

In [ ]:

from sklearn.metrics import roc_curve, auc
"""
ROC score
"""
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic curve')
plt.show()
print('AUC: %f' % roc_auc)

In [ ]:
plt.figure()
plt.plot(model.history.history['acc'])
plt.plot(model.history.history['val_acc'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

model for testing associations. 
it's the same model as before but this time the input layer is the merged convolutions. so you can fire filters as desired. 

In [ ]:
merged_conv = Input(shape=(1,24),name='input_for_association_test')
deeplayer1_2 = model.get_layer('deeplayer1')(merged_conv)
deeplayer2_2 = model.get_layer('deeplayer2')(deeplayer1_2)

prob_association_2 = model.get_layer('logistic_regression')(deeplayer2_2)
model2 = Model(inputs=merged_conv,outputs=prob_association_2)

In [ ]:
association_predictor = np.zeros((24*24,1,24))
c = 0
for i in range(24):
    for j in range(24):
        association_predictor[c][0][i]=1
        association_predictor[c][0][j]=1
        c += 1

In [ ]:
y_score = model2.predict(association_predictor)

In [ ]:
association_matrix = np.zeros((24,24))
c = 0
for i in range(24):
    for j in range(24):
        association_matrix[i][j]=y_score[c]
        c += 1

In [ ]:
plt.figure()
plt.imshow(association_matrix,interpolation='none',cmap='gray')
plt.axvline(11.5,color='r')
plt.axhline(11.5,color='r')
plt.show()